# Tutorial notebook for dinucleotide forces computation

We will take an example genome and we will use it as working example to test the function of the scripts.

In [1]:
# load the genome - this is an Influenza H5N1 PB2 segment, strain used: A/Anhui/1/2005
sequence = "ATGGAGAGAATAAAAGAATTAAGGGATCTAATGTCACAGTCCCGCACTCGCGAGATACTAACAAAAACCACTGTGGACCATATGGCCATAATCAAGAAGTACACATCAGGAAGACAAGAGAAGAACCCTGCTCTCAGAATGAAATGGATGATGGCAATGAAATATCCAATCACAGCGGACAAGAGAATAACAGAGATGATTCCTGAAAGGAATGAACAAGGGCAGACGCTCTGGAGCAAGACAAATGATGCCGGATCGGACAGGTTGATGGTGTCTCCCTTAGCTGTAACTTGGTGGAATAGGAATGGGCCGACGACAAGTGCAGTCCATTATCCAAAGGTTTACAAAACATACTTTGAGAAGGCTGAAAGGCTAAAACATGGAACCTTCGGTCCCGTCCATTTTCGAAACCAAGTTAAAATACGCCGCCGAGTTGATATAAATCCTGGCCATGCAGATCTCAGTGCTAAAGAAGCACAAGATGTCATCATGGAGGTCGTTTTCCCAAATGAAGTGGGAGCTAGAATATTGACATCAGAGTCACAATTGACAATAACGAAAGAGAAGAAAGAAGAGCTCCAAGATTGTAAGATTGCTCCCTTAATGGTTGCATACATGTTGGAAAGGGAACTGGTCCGCAAAACCAGATTCCTACCGGTAGCAAGCGGAACAAGCAGTGTGTACATTGAGGTATTGCATTTGACTCAAGGGACCTGCTGGGAACAGATGTACACTCCAGGCGGAGAAGTGAGAAACGACGATGTTGACCAGAGTTTGATCATCGCTGCCAGAAACATTGTTAGGAGAGCAACGGTATCAGCGGATCCACTGGCATCACTGCTGGAGATGTGTCACAGCACACAAATTGGTGGGATAAGGATGGTGGACATCCTTAGGCAAAACCCAACTGAGGAACAAGCTGTGGGTATATGCAAAGCAGCAATGGGTCTGAGGATCAGTTCATCCTTTAGCTTTGGAGGCTTCACTTTCAAAAGAACAAGTGGATCATCCGTCACGAAGGAAGAGGAAGTGCTTACAGGCAACCTCCAAACATTGAAAATAAGAGTACATGAGGGGTATGAAGAGTTCACAATGGTTGGACGGAGGGCAACAGCTATCCTGAGGAAAGCAACTAGAAGGCTGATTCAGTTGATAGTAAGTGGAAGAGACGAACAATCAATCGCTGAGGCAATCATTGTAGCAATGGTGTTCTCACAGGAGGATTGCATGATAAAGGCAGTCCGGGGCGATTTGAATTTCGTAAACAGAGCAAACCAAAGATTAAACCCCATGCATCAACTCCTGAGACATTTTCAAAAGGACGCAAAAGTGCTATTTCAGAATTGGGGAATTGAACCCATTGATAATGTCATGGGGATGATCGGAATATTACCTGACCTGACTCCCAGCACAGAAATGTCACTGAGAAGAGTAAGAGTTAGTAAAGTGGGAGTGGATGAATATTCCAGCACTGAGAGAGTAATTGTAAGTATTGACCGTTTCTTAAGGGTTCGAGATCAGCGGGGGAACGTACTCTTATCTCCCGAAGAGGTCAGCGAAACCCAGGGAACAGAGAAATTGACAATAACATATTCATCATCAATGATGTGGGAAATCAACGGTCCTGAGTCAGTGCTTGTTAACACCTATCAATGGATCATCAGAAACTGGGAAACTGTGAAGATTCAATGGTCTCAAGACCCCACGATGCTGTACAATAAGATGGAGTTTGAACCGTTCCAATCCTTGGTACCTAAGGCTGCCAGAGGTCAATACAGTGGATTTGTGAGAACACTATTCCAACAAATGCGTGACGTACTGGGGACATTTGATACTGTCCAGATAATAAAGCTGCTACCATTTGCAGCAGCCCCACCAGAGCAGAGCAGAATGCAGTTTTCTTCTCTAACTGTGAATGTGAGAGGCTCAGGAATGAGAATACTCGTAAGGGGCAATTCCCCTGTGTTCAACTACAATAAGGCAACCAAAAGGCTTACCGTTCTTGGAAAGGACGCAGGTGCATTAACAGAGGATCCAGATGAGGGGACAACCGGAGTGGAGTCTGCAGTACTGAGGGAATTCCTAATTCTAGGCAAGGAGGACAAAAGATATGGACCAGCATTGAGTATCAATGAACTGAGCAACCTTGCGAAAGGGGAGAAAGCTAATGTGCTGATAGGACAAGGAGACGTGGTGTTGGTAATGAAACGGAAACGGGACTCTAGCATACTTACTGACAGCCAGACAGCGACCAAAAGAATTCGGATGGCCATCAATTAG";

## Non-coding forces

In [2]:
package_path = "./"
!in(package_path, LOAD_PATH) && push!(LOAD_PATH, package_path)
using NoncodingForces_v2_1

┌ Info: Precompiling NoncodingForces_v2_1 [top-level]
└ @ Base loading.jl:1423


Let's start by computing the force on the CpG motif along the full genome:

In [3]:
motifs = ["CG"]

NoncodingForces_v2_1.DimerForce(sequence, motifs)

Dict{String, Float64} with 5 entries:
  "A"  => -1.1769
  "T"  => -1.61568
  "C"  => -1.52965
  "CG" => -0.995176
  "G"  => -1.28591

When no nucleotide biases (frequencies) are specified, as above, the corresponding fields are inferred together with the forces. 
Notice that $\sum_n e^{h_n} = 1$, where $n \in \{A,C,G,T\}$. This is the result of a gauge choice, because in general the model probabilities are invariant under the transformation
$$ h_n \to h_n+K, $$
which can then be used to make them interpretable as the logarithm of a frequency.

A user-specified bias can be given, and in this case fields are not inferred:

In [4]:
nt_bias = [0.25, 0.25, 0.25, 0.25] # probs for A, C, G, T
motifs = ["CG"]

NoncodingForces_v2_1.DimerForce(sequence, motifs; freqs=nt_bias)

Dict{String, Float64} with 1 entry:
  "CG" => -1.04814

The script easily allow to compute forces on two or more dinucleotides:

In [5]:
motifs = ["CG", "TA"]

NoncodingForces_v2_1.DimerForce(sequence, motifs)

Dict{String, Float64} with 6 entries:
  "A"  => -1.11638
  "T"  => -1.48577
  "C"  => -1.63225
  "CG" => -0.91146
  "G"  => -1.38341
  "TA" => -0.762974

Notice how the CpG force changed in the 3 cases, due to the fact that different motifs (as well as fields) interact. 
When the full set of dinucleotides is inferred, the system of equations solved to obtain forces is underdetermined. This means that a gauge choice must be made, for instance by setting some forces to zero by passing less than 16 dinucleotides as motifs.
If such a choise is not made, the script does it, as follows:

In [13]:
alphabet = ["A", "C", "G", "T"]
motifs = [a*b for a in alphabet for b in alphabet]

NoncodingForces_v2_1.DimerForce(sequence, motifs)

Dict{String, Float64} with 20 entries:
  "T"  => -1.51332
  "C"  => -1.58152
  "CC" => 0.0
  "GC" => -0.0745035
  "GG" => 0.0
  "CG" => -0.923862
  "AT" => 0.0
  "A"  => -1.15323
  "CA" => 0.244622
  "TG" => 0.0349993
  "TA" => -0.508575
  "GT" => 0.0
  "G"  => -1.35269
  "GA" => 0.319795
  "TT" => 0.0
  "AC" => -0.152995
  "CT" => 0.0
  "AA" => 0.0
  "AG" => -0.178602
  "TC" => 0.0532732

In particular, the gauge chosen is so that:
- the exponential of the fields sum to 1;
- the forces for dinucleotides of the form NN and those of the form NT are put to zero.

Notice that the script allows for flexible choices: if not all the dinucleotides are given, the gauge is always chosen such that the maximum possible number of the nucleotides of the form NN and NT have forces equal to zero:

In [7]:
motifs = ["AC", "AG", "AT", "CA", "GA", "TA"]
NoncodingForces_v2_1.DimerForce(sequence, motifs)

Dict{String, Float64} with 10 entries:
  "AC" => -0.151479
  "AT" => 0.0
  "A"  => -1.10053
  "T"  => -1.3595
  "C"  => -1.75552
  "AG" => 0.00905578
  "CA" => 0.45918
  "G"  => -1.43675
  "TA" => -0.617891
  "GA" => 0.258216

In [8]:
motifs = [
 "AC",
 "AG",
 "AT",
 "CA",

 "CG",
 "CT",
 "GA",
 "GC",

 "GT",
 "TA",
 "TC",
 "TG",
]
NoncodingForces_v2_1.DimerForce(sequence, motifs)

Dict{String, Float64} with 16 entries:
  "T"  => -1.52067
  "C"  => -1.58192
  "GC" => -0.0729472
  "CG" => -0.921501
  "AT" => 0.0
  "A"  => -1.14765
  "CA" => 0.238663
  "TG" => 0.0479085
  "G"  => -1.35296
  "TA" => -0.503996
  "GT" => 0.0
  "GA" => 0.313334
  "AC" => -0.151742
  "CT" => 0.0
  "AG" => -0.177036
  "TC" => 0.0658684

Notice that when dinucleotides are not given, this is equivalent to fix to 0 their forces. Depending on which dinucleotides are not given, this might or not be a specific gauge - in other words, it might or not result in an equivalent model.
For instance, the following three cells result in equivalent models (although the third has different parameters because it is in another gauge), while the fourth does not:

In [9]:
motifs = ["AC", "AG", "AT", "CA", "GA", "TA"]
NoncodingForces_v2_1.DimerForce(sequence, motifs)

Dict{String, Float64} with 10 entries:
  "AC" => -0.151479
  "AT" => 0.0
  "A"  => -1.10053
  "T"  => -1.3595
  "C"  => -1.75552
  "AG" => 0.00905578
  "CA" => 0.45918
  "G"  => -1.43675
  "TA" => -0.617891
  "GA" => 0.258216

In [10]:
motifs = ["AC", "AG", "CA", "GA", "TA"]
NoncodingForces_v2_1.DimerForce(sequence, motifs)

Dict{String, Float64} with 9 entries:
  "AC" => -0.151479
  "A"  => -1.10053
  "T"  => -1.3595
  "C"  => -1.75552
  "AG" => 0.00905578
  "CA" => 0.45918
  "G"  => -1.43675
  "TA" => -0.617891
  "GA" => 0.258216

In [11]:
motifs = ["AG", "AT", "CA", "GA", "TA"]
NoncodingForces_v2_1.DimerForce(sequence, motifs)

Dict{String, Float64} with 9 entries:
  "AT" => 0.170798
  "A"  => -1.10038
  "T"  => -1.36235
  "C"  => -1.75204
  "AG" => 0.17066
  "CA" => 0.297683
  "G"  => -1.43641
  "TA" => -0.779388
  "GA" => 0.0967191

In [12]:
motifs = ["AG", "CA", "GA", "TA"]
NoncodingForces_v2_1.DimerForce(sequence, motifs)

Dict{String, Float64} with 8 entries:
  "A"  => -1.10231
  "T"  => -1.33747
  "C"  => -1.78327
  "AG" => 0.0793553
  "CA" => 0.388927
  "G"  => -1.43844
  "TA" => -0.688144
  "GA" => 0.187962

In some cases (notably when all the dinucleotides are given), fixing fields is equivalent to fix (part of) a gauge, so the model is independendent on this choice (although the values of the inferred parameters depend on that).

Given a sequence, the package allows to compute easily its energy

In [13]:
tf = NoncodingForces_v2_1.DimerForce(sequence, motifs)
NoncodingForces_v2_1.compute_energy(sequence, tf)

-3060.7803535779112

and its log-likelihood (energy minus log of partition function)

In [14]:
NoncodingForces_v2_1.compute_loglikelihood(sequence, tf)

-3077.2773914820596

To make sense of this large negative log-likelihood, consider that in the uniform case we have $4^L$ sequences, so the log-likelihood of each of them would be:

In [15]:
- length(sequence) * log(4)

-3160.7511433533505

Notice that the function `compute_loglikelihood` computes the partition function each time it is runned. To avoid this, the log of the partition function can be passed directly:

In [19]:
@time begin
NoncodingForces_v2_1.compute_loglikelihood(sequence, tf)
end

  0.000334 seconds (4.22 k allocations: 379.297 KiB)


-3077.2773914820596

In [20]:
ks = keys(tf)
k1 = [k for k in ks if length(k)==1]
k2 = [k for k in ks if length(k)==2]
fields = [tf[k] for k in k1]
forces = [tf[k] for k in k2]
L = length(sequence)
logZ = NoncodingForces_v2_1.eval_log_Z(fields, forces, k2, L)

43.95270666581519

In [21]:
@time begin
NoncodingForces_v2_1.compute_loglikelihood(sequence, tf; logZ=logZ)
end

  0.000182 seconds (1.44 k allocations: 89.922 KiB)


-3104.7330602437264

# From here on, it is work in progress!!!

In [ ]:
##########################################################################
##########################################################################

In [ ]:
# this below seems not to work well...

Using "add_pseudocount = true", a single pseudocount is added to the number of each motif inferred (nucleotide or di-nucleotide):

In [22]:
short_sequence = sequence[1:500]

"ATGGAGAGAATAAAAGAATTAAGGGATCTAATGTCACAGTCCCGCACTCGCGAGATACTAACAAAAACCACTGTGGACCATATGGCCATAATCAAGAAGTACACATCAGGAAGACAAGAGAAGAACCCTGCTCTCAGAATGAAATGGATGATGGCAATGAAATATCCAATCACAGCGGACAAGAGAATAACAGAGATGATTCCTGAAAGGAATGAACAAGGGCAGACGCTCTGGAGCAAGACAAATGATGCCGGATCGGACAGGTTGATGGTGTCTCCCTTAGCTGTAACTTGGTGGAATAGGAATGGGCCGACGACAAGTGCAGTCCATTATCCAAAGGTTTACAAAACATACTTTGAGAAGGCTGAAAGGCTAAAACATGGAACCTTCGGTCCCGTCCATTTTCGAAACCAAGTTAAAATACGCCGCCGAGTTGATATAAATCCTGGCCATGCAGATCTCAGTGCTAAAGAAGCACAAGATGTCATCATGGAGGTCGT"

In [24]:
alphabet = ["A", "C", "G", "T"]
motifs = [a*b for a in alphabet for b in alphabet]
NoncodingForces_v2_1.DimerForce(short_sequence, motifs; add_pseudocount=false)

Dict{String, Float64} with 20 entries:
  "T"  => -1.78999
  "C"  => -1.46096
  "CC" => 0.0
  "GC" => -0.0881668
  "GG" => 0.0
  "CG" => -0.673973
  "AT" => 0.0
  "A"  => -1.02452
  "CA" => 0.201549
  "TG" => 0.285663
  "TA" => -0.190423
  "GT" => 0.0
  "G"  => -1.41859
  "GA" => 0.470057
  "TT" => 0.0
  "AC" => -0.497816
  "CT" => 0.0
  "AA" => 0.0
  "AG" => -0.420048
  "TC" => 0.323648

In [23]:
alphabet = ["A", "C", "G", "T"]
motifs = [a*b for a in alphabet for b in alphabet]
NoncodingForces_v2_1.DimerForce(short_sequence, motifs; add_pseudocount=true)

Dict{String, Float64} with 20 entries:
  "T"  => -3.3792
  "C"  => -1.30829
  "CC" => 0.0
  "GC" => -0.0826845
  "GG" => 0.0
  "CG" => -0.645923
  "AT" => 0.0
  "A"  => -0.881581
  "CA" => 0.191599
  "TG" => 2.18227
  "TA" => 1.71958
  "GT" => 0.0
  "G"  => -1.26757
  "GA" => 0.455001
  "TT" => 0.0
  "AC" => -0.48536
  "CT" => 0.0
  "AA" => 0.0
  "AG" => -0.409147
  "TC" => 2.21938

In [25]:
short_sequence = sequence[1:200]
alphabet = ["A", "C", "G", "T"]
motifs = [a*b for a in alphabet for b in alphabet]
NoncodingForces_v2_1.DimerForce(short_sequence, motifs; add_pseudocount=true)

Dict{String, Float64} with 20 entries:
  "T"  => NaN
  "C"  => NaN
  "CC" => NaN
  "GC" => NaN
  "GG" => NaN
  "CG" => NaN
  "AT" => NaN
  "A"  => NaN
  "CA" => NaN
  "TG" => NaN
  "TA" => NaN
  "GT" => NaN
  "G"  => NaN
  "GA" => NaN
  "TT" => 0.0
  "AC" => NaN
  "CT" => NaN
  "AA" => NaN
  "AG" => NaN
  "TC" => NaN

In [26]:
short_sequence = sequence[1:100]
alphabet = ["A", "C", "G", "T"]
motifs = [a*b for a in alphabet for b in alphabet]
NoncodingForces_v2_1.DimerForce(short_sequence, motifs; add_pseudocount=true)

LoadError: LinearAlgebra.SingularException(1)

## Coding forces

TODO